In [1]:
import os
import pandas as pd
import numpy as np
import json
import folium
import glob
import csv
import string

For the second task we have downloaded from the amstat website a dataset that includes "Unemployed and people without a job for various criteria" as registered in September 2017. We assume that .... 
The data was saved in the excel format and contains the information we are interested to, specifically:
* Registered unemployed
* Registered job seekers
* Job seekers with a job

We drop the unuseful information and rename properly the columns

# Q2 : Canton wise unemployment rate

In [2]:
#Import
ch_geo = os.path.join('./topojson/', 'ch-cantons.topojson.json')
ch_rate = os.path.join('./ch_rate.xlsx')

#Reading
ch_rate_data = pd.read_excel(ch_rate, index=False)

#Clean 1st dataset
ch_rate_data = ch_rate_data.iloc[1:,:]

In [23]:
ch_rate_data.head()

,Kanton,rate_ue_2002,reg_ue_2002,js_with_job_2002,rate_ue_2003,reg_ue_2003,js_with_job_2003,rate_ue_2004,reg_ue_2004,js_with_job_2004,...,js_with_job_2013,rate_ue_2014,reg_ue_2014,js_with_job_2014,rate_ue_2015,reg_ue_2015,js_with_job_2015,rate_ue_2016,reg_ue_2016,js_with_job_2016
1,Zürich,2.991667,21595.750000,7283.416667,4.516667,32574.333333,9444.666667,4.483333,32402.166667,10368.500000,...,5790.833333,3.175000,26013.166667,5923.583333,3.416667,27985.333333,6232.916667,3.650000,30083.916667,6702.000000
2,Bern,1.750000,9162.916667,5155.666667,2.833333,14860.416667,6651.333333,2.916667,15212.583333,7452.500000,...,4514.916667,2.350000,13189.333333,4866.000000,2.500000,14116.416667,4922.666667,2.708333,15283.166667,4801.083333
3,Luzern,2.000000,3780.000000,1560.583333,3.116667,5881.666667,2334.833333,3.141667,5955.333333,2676.666667,...,2489.750000,1.866667,4147.083333,2435.916667,1.975000,4383.333333,2643.916667,2.058333,4601.750000,2864.833333
4,Uri,0.716667,124.416667,169.500000,1.066667,188.000000,181.500000,1.183333,208.833333,200.666667,...,182.750000,1.050000,202.916667,148.083333,1.033333,199.666667,163.833333,1.041667,201.500000,161.916667
5,Schwyz,1.225000,878.166667,730.500000,2.108333,1490.500000,1034.250000,2.283333,1630.333333,1019.666667,...,773.000000,1.525000,1344.083333,794.250000,1.583333,1388.333333,758.666667,1.766667,1537.166667,752.416667


In [4]:
for year in np.arange(0,15,1):
    rate_data = pd.DataFrame({'rate_ue_{}'.format(year+2002): ch_rate_data.iloc[:,(1+36*year):(1+36*(year+1)-1):3].apply(pd.to_numeric).mean(axis=1)})
    reg_ue_data = pd.DataFrame({'reg_ue_{}'.format(year+2002): ch_rate_data.iloc[:,(2+36*year):(2+36*(year+1)-1):3].apply(pd.to_numeric).mean(axis=1)})
    js_with_job = pd.DataFrame({'js_with_job_{}'.format(year+2002): ch_rate_data.iloc[:,(3+36*year):(3+36*(year+1)-1):3].apply(pd.to_numeric).mean(axis=1)})
    ch_rate_data = pd.concat((ch_rate_data,rate_data,reg_ue_data,js_with_job),axis=1)

In [5]:
beginning_of_dataset = ch_rate_data.iloc[:,:1]
end_of_dataset = ch_rate_data.iloc[:,-45:]
ch_rate_data = pd.concat((beginning_of_dataset,end_of_dataset),axis=1)

In [22]:
ch_rate_data.head()

,Kanton,rate_ue_2002,reg_ue_2002,js_with_job_2002,rate_ue_2003,reg_ue_2003,js_with_job_2003,rate_ue_2004,reg_ue_2004,js_with_job_2004,...,js_with_job_2013,rate_ue_2014,reg_ue_2014,js_with_job_2014,rate_ue_2015,reg_ue_2015,js_with_job_2015,rate_ue_2016,reg_ue_2016,js_with_job_2016
1,Zürich,2.991667,21595.750000,7283.416667,4.516667,32574.333333,9444.666667,4.483333,32402.166667,10368.500000,...,5790.833333,3.175000,26013.166667,5923.583333,3.416667,27985.333333,6232.916667,3.650000,30083.916667,6702.000000
2,Bern,1.750000,9162.916667,5155.666667,2.833333,14860.416667,6651.333333,2.916667,15212.583333,7452.500000,...,4514.916667,2.350000,13189.333333,4866.000000,2.500000,14116.416667,4922.666667,2.708333,15283.166667,4801.083333
3,Luzern,2.000000,3780.000000,1560.583333,3.116667,5881.666667,2334.833333,3.141667,5955.333333,2676.666667,...,2489.750000,1.866667,4147.083333,2435.916667,1.975000,4383.333333,2643.916667,2.058333,4601.750000,2864.833333
4,Uri,0.716667,124.416667,169.500000,1.066667,188.000000,181.500000,1.183333,208.833333,200.666667,...,182.750000,1.050000,202.916667,148.083333,1.033333,199.666667,163.833333,1.041667,201.500000,161.916667
5,Schwyz,1.225000,878.166667,730.500000,2.108333,1490.500000,1034.250000,2.283333,1630.333333,1019.666667,...,773.000000,1.525000,1344.083333,794.250000,1.583333,1388.333333,758.666667,1.766667,1537.166667,752.416667


 For our analysis we choose to keep only the fraction of people who strictly belong to the definition of unemployment of the Bureau of Labor Statistics https://www.bls.gov/cps/cps_htgm.htm: "People who are jobless, looking for a job, and available for work". That is, the number of registered job seekers minus the ones who have already a job. We can easily see that this number corresponds to the registered unemployed people. It is important to check this difference to exclude the possbility that the "registered unemployed" class includes jobless people that are not looking for a job (and that thus don't belong to our definition of unemployment).

Moreover, we are interested in the unemployment rate, which is the number of unemployed people divided by the number of people in the labor force. 
The labor force is the sum of employed and unemplyed people. Even though this information is not provided in the dataset, we assume that the "Unemployment rate" has been computed from the "Registered unemployed" data and thus we will use this information for our analysis.

Our next step is to use folium to visualize Swiss cantons and assign the corresponding unemployment rates.
Notice that we have downloaded the data in German in order to better match the names of the cantons with the ones provided in the geo data.

In [20]:
#Loading
ch_geo = open('geojson/switzerland.geojson')
ch_geo = ch_geo.read()
ch_geo = json.loads(ch_geo)

#Assign geoJson
ch_map = folium.Map(location=[46.8,8.1], zoom_start=7.5)
folium.GeoJson(ch_geo, name='ch_geo').add_to(ch_map)

#Visualize
ch_map

In [21]:
ch_map.choropleth(geo_data=ch_geo, data=ch_rate_data,
             columns=['Kanton', 'rate_ue_2008'],
             key_on='feature.properties.name',
             fill_color='OrRd', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Unemployment rate (%)')
folium.LayerControl().add_to(ch_map)
ch_map

## Calculate the same as above by removing the job seekers with jobs

# TBD LATER : Calculate the same but by using other categories like maybe young people

## Calculate rate by using `ch_nationality` and `ch_age datasets`, use the same preprocessing as above (rates are available)

# IMP : 
You have to find out the total number of each category by using the `ch_total` dataset and calculate the rates like that. 

By using the language region information, calculate an average unemployment rate across the two major linguistic regions and comment on the Röstigraben

Comment on the general trends of all the previous questions. Example : in times of crisis, foreign workers are usually kicked out. 